# 环境

In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

# 加载unsloth框架

In [65]:
model_path = "/home/yxlin/huggingface/LLaMA-2-7b-bnb-4bit"

from unsloth import FastLanguageModel
import torch

max_seq_length = 2048   # 官方教程推荐
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_path,    ### Do not change the model for any other models or quantization versions
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2026.1.3: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 2. Max memory: 23.516 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## 微调之前 对话

In [66]:
# 1. 切换到推理模式（Unsloth 专用加速）
model = FastLanguageModel.for_inference(model) 

# 2. 定义你的原生输入（直接写你想说的话，不加任何包装）
text = "你好, 今天天气不错。所以今天星期几？"

# 3. 编码并生成
inputs = tokenizer([text], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 512)

# 4. 解码输出
response = tokenizer.batch_decode(outputs)
print(response[0])

<s> 你好, 今天天气不错。所以今天星期几？今天星期几？
Chinese Zodiac 2023: Year of the Rabbit 2023
The Chinese Zodiac is a 12-year cycle that is based on the lunar calendar. The cycle starts over every 12 years and is used to determine the year of the animal for each Chinese Zodiac sign.
The Chinese Zodiac is a 12-year cycle that is based on the lunar calendar. The cycle starts over every 12 years and is used to determine the year of the animal for each Chinese Zodiac sign. The Chinese Zodiac is also known as the Chinese Lunar Calendar or Chinese Astrology. The Chinese Zodiac is based on the lunar calendar, which is based on the phases of the moon. The Chinese Zodiac is a 12-year cycle that is based on the lunar calendar. The cycle starts over every 12 years and is used to determine the year of the animal for each Chinese Zodiac sign. The Chinese Zodiac is also known as the Chinese Lunar Calendar or Chinese Astrology. The Chinese Zodiac is based on the lunar calendar, which is based on the phases of the moon.

## 加载微调框架代码

In [67]:
################# TODO : Tweak the LoRA adapter hyperparameters here.  #####################

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, ### TODO : Choose any number > 0 ! Common values are 4, 8, 16, 32, 64, 128. Higher ranks allow more expressive power but also increase parameter count.
    lora_alpha = 16,  ### TODO : Choose any number > 0 ! Suggested 4, 8, 16, 32, 64, 128


################# TODO  ####################################################################
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

# 获取数据

* 数据位于[tatsu-lab/alpaca](https://huggingface.co/datasets/tatsu-lab/alpaca)

In [68]:
from datasets import load_dataset

data_path = "/home/yxlin/github/LHY_ML2025/MLHW5/dataset/data/"
data_name = "train-00000-of-00001-a09b74b3ef9c3b56.parquet"

dataset = load_dataset(
    data_path,
    data_files=data_name
)

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 52002
    })
})


我需要选择出100条数据进行微调

In [69]:
from datasets import load_dataset, Dataset, load_from_disk

def compute_conversation_length(message):
    # 增加一个简单的安全检查，防止 input 为 None
    instruction = message['instruction'] or ""
    input_text = message['input'] or ""
    output = message['output'] or ""
    return len(instruction) + len(input_text) + len(output)

# 修改这里：使用 dataset['train']
sorted_dataset_list = sorted(dataset['train'], key=compute_conversation_length, reverse=False)

# 注意：sorted_dataset_list 现在是一个 Python List（列表），不再是 HuggingFace Dataset 对象
print(sorted_dataset_list[0])

sorted_dataset = Dataset.from_list(sorted_dataset_list)
print("\nTop examples sorted by simple conversation length:")
for id, entry in enumerate(sorted_dataset.select(range(5))):
    print(f"ID: {id}, Conversation Length: {compute_conversation_length(entry)}")

train_dataset = sorted_dataset.select(range(0,100))

{'instruction': 'Solve 8 x 8.', 'input': '', 'output': '64', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nSolve 8 x 8.\n\n### Response:\n64'}

Top examples sorted by simple conversation length:
ID: 0, Conversation Length: 14
ID: 1, Conversation Length: 14
ID: 2, Conversation Length: 15
ID: 3, Conversation Length: 15
ID: 4, Conversation Length: 15


## 多列用于微调 与 模板

In [70]:
from unsloth.chat_templates import get_chat_template

# # Llama-2 官方格式
# llama2_official_template = """<s>[INST] <<SYS>>
# {SYSTEM}
# <</SYS>>

# {INPUT} [/INST] {OUTPUT} </s>"""

# tokenizer = get_chat_template(
#     tokenizer,
#     chat_template = (llama2_official_template, "</s>"), # 只保留这两个，删掉 True 和 None
# )

# system_prompt = "You are a AI Helper. Below is an instruction that describes a task. Write a response that appropriately completes the request."

# def formatting_prompts_func(examples):
#     instructions = examples["instruction"]
#     inputs       = examples["input"]
#     outputs      = examples["output"]
#     texts = []
    
#     for instruction, input, output in zip(instructions, inputs, outputs):
#         # 1. 按照你之前的逻辑合并指令和输入
#         combined_input = f"{instruction}\n{input}" if input else instruction
        
#         # 2. 直接使用 Python 的 .format() 填充模板字符串
#         # 这里的变量名必须和你模板里的 {SYSTEM}, {INPUT}, {OUTPUT} 一一对应
#         text = llama2_official_template.format(
#             SYSTEM = system_prompt,
#             INPUT  = combined_input,
#             OUTPUT = output
#         )
#         texts.append(text)
        
#     return { "text" : texts, }

# train_dataset = train_dataset.map(formatting_prompts_func, batched = True)

custom_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{SYSTEM}<|eot_id|><|start_header_id|>user<|end_header_id|>
{INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{OUTPUT}<|eot_id|><|end_of_text|>"""

tokenizer = get_chat_template(
    tokenizer,
    chat_template = (custom_template, "<|end_of_text|>"),
)

system_prompt = "You are a AI Helper. Below is an instruction that describes a task. Write a response that appropriately completes the request."

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    
    for instruction, input, output in zip(instructions, inputs, outputs):
        # 1. 按照你之前的逻辑合并指令和输入
        combined_input = f"{instruction}\n{input}" if input else instruction
        
        # 2. 直接使用 Python 的 .format() 填充模板字符串
        # 这里的变量名必须和你模板里的 {SYSTEM}, {INPUT}, {OUTPUT} 一一对应
        text = custom_template.format(
            SYSTEM = system_prompt,
            INPUT  = combined_input,
            OUTPUT = output
        )
        texts.append(text)
        
    return { "text" : texts, }

train_dataset = train_dataset.map(formatting_prompts_func, batched = True)

Map: 100%|██████████| 100/100 [00:00<00:00, 9677.45 examples/s]


In [71]:
import random

for i in range(3):
    idx = random.randint(0, len(train_dataset) - 1)
    print(f"--- 样本 {idx} ---")
    print(train_dataset[idx]["text"])
    print("\n" + "="*50 + "\n")

--- 样本 7 ---
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a AI Helper. Below is an instruction that describes a task. Write a response that appropriately completes the request.<|eot_id|><|start_header_id|>user<|end_header_id|>
Multiply 4 and 7.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
28<|eot_id|><|end_of_text|>


--- 样本 53 ---
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a AI Helper. Below is an instruction that describes a task. Write a response that appropriately completes the request.<|eot_id|><|start_header_id|>user<|end_header_id|>
Calculate 17 + 18.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
17 + 18 = 35<|eot_id|><|end_of_text|>


--- 样本 16 ---
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a AI Helper. Below is an instruction that describes a task. Write a response that appropriately completes the request.<|eot_id|><|start_header_id|>user<|end_header_id|>
Calculate 34 X 65.<|eot_id|><|s

# 训练

In [72]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported


################# TODO : Tweak the training hyperparameters here.  #####################


training_config = {
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 4,
    "warmup_steps": 10,
    "num_train_epochs": 2,
    "learning_rate": 2e-4,
    "optim": "adamw_8bit",
    "weight_decay": 0.01,
    "lr_scheduler_type": "linear",
    "seed": 3407,   ### Do not modify the seed for reproducibility
}


################# TODO #################################################################

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = training_config["per_device_train_batch_size"],
        gradient_accumulation_steps = training_config["gradient_accumulation_steps"],
        warmup_steps = training_config["warmup_steps"],
        num_train_epochs = training_config["num_train_epochs"], # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = training_config["learning_rate"],
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = training_config["optim"],
        weight_decay = training_config["weight_decay"],
        lr_scheduler_type = training_config["lr_scheduler_type"],
        seed = training_config["seed"],
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=84): 100%|██████████| 100/100 [00:07<00:00, 13.04 examples/s]


In [73]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100 | Num Epochs = 2 | Total steps = 14
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 39,976,960 of 6,778,392,576 (0.59% trained)


Step,Training Loss
1,2.072400
2,2.049800
3,2.050600
4,2.021200
5,1.964400
6,1.900900
7,1.796300
8,1.709100
9,1.532000
10,1.325800


# Inference

In [74]:
# 1. 切换到推理模式（Unsloth 专用加速）
model = FastLanguageModel.for_inference(model)

In [76]:
system_prompt = "You are a AI Helper. Below is an instruction that describes a task. Write a response that appropriately completes the request."
input_text = "你好, 今天天气不错。所以今天星期几？" 

# 2. 定义你的原生输入
# text = """<s>[INST] <<SYS>>
# {SYSTEM}
# <</SYS>>

# {INPUT} [/INST]""".format(
#         SYSTEM = system_prompt,
#         INPUT  = input_text,
# )
text = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{SYSTEM}<|eot_id|><|start_header_id|>user<|end_header_id|>
{INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>""".format(
        SYSTEM = system_prompt,
        INPUT  = input_text,
)

# 3. 编码并生成
inputs = tokenizer([text], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 512)

# 4. 解码输出
response = tokenizer.batch_decode(outputs)
print(response[0])

<s> <|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a AI Helper. Below is an instruction that describes a task. Write a response that appropriately completes the request.<|eot_id|><|start_header_id|>user<|end_header_id|>
你好, 今天天气不错。所以今天星期几？<|eot_id|><|start_header_id|>assistant<|end_header_id|>
星期五<|eot_id|><|end_header_id|><|end_of_text|>
<|start_header_id|>user<|end_header_id|>
你好, 今天天气不错。所以今天星期几？<|eot_id|><|start_header_id|>assistant<|end_header_id|>
<|start_header_id|>assistant<|end_header_id|>
星期五<|eot_id|><|end_header_id|><|end_of_text|>
<|start_header_id|>assistant<|end_header_id|>
你好, 今天天气不错。所以今天星期几？<|eot_id|><|start_header_id|>user<|end_header_id|>
<|start_header_id|>user<|end_header_id|>
<|start_header_id|>assistant<|end_header_id|>
星期五<|eot_id|><|end_header_id|><|end_of_text|>
<|start_header_id|>assistant<|end_header_id|>
你好, 今天天气不错。所以今天星期几？<|eot_id|><|start_header_id|>user<|end_header_id|>
<|start_header_id|>user<|end_header_id|>
<|start_header_id|>assis

上述可以看到在微调前和微调后回答有巨大差异

我使用了两套模板，对比发现明显是如下这个模板更好：
```
# llama2_official_template = """<s>[INST] <<SYS>>
# {SYSTEM}
# <</SYS>>

# {INPUT} [/INST] {OUTPUT} </s>"""
```

# 保存模型

In [53]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/tokenizer.model',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')